In [1]:
import numpy as np
import pandas as pd
import requests
import re
import ftplib
import xarray as xr
import os
import appdirs
import time
from datetime import datetime
from io import StringIO
import matplotlib.pyplot as plt
from dateutil import relativedelta

In [6]:
pre_name = 'Baseline_ilt_extended'
plname = 'Trend_' + pre_name
tag = ''
predictors = pd.read_csv('/home/poyraden/MLR_Uccle/Files/Extended_ilt.csv')
# For DeBilt
predictors = predictors.loc['1992-11-01':'2018-12-01']
predictors

,Unnamed: 0,linear_pre,linear_post,post_const,pre_const,gap_const,enso,qboA,qboB,solar
200,1985-09,-1.133494,0.0,0.0,1.0,0.0,-0.535,0.519112,-1.026784,-1.088319
201,1985-10,-1.125280,0.0,0.0,1.0,0.0,-0.148,0.149196,-1.143714,-1.002959


In [5]:
from LOTUS_regression.regression import mzm_regression


uccle = pd.read_csv('/home/poyraden/MLR_Uccle/Files/DeBilt_1km_monthlymean_deseas.csv')
print(list(uccle))
#dates = pd.date_range(start='1992-11', end='2018-12', freq = 'MS')

uccle.rename(columns={'Unnamed: 0':'date'}, inplace=True)
#uccle['date'] =  dates
pd.to_datetime(uccle['date'], format='%Y-%m')
uccle.set_index('date', inplace=True)

print(list(uccle))
alt = [''] * 36
alt_ds = [''] * 36

# uc = pd.DataFrame()
# uct = pd.DataFrame()
uc = {}
uct = {}
ucm = {}

regression_output = [0] * 36
uX = [0] * 36
uY = [0] * 36
param_list = [0] * 36
error_list = [0] * 36
ut = [0] * 36

trend_pre = [0] * 36
trend_pre_err = [0] * 36
trend_post = [0] * 36
trend_post_err = [0] * 36

mY = []

uct_pre = {}
uct_post = {}

mean_pre = [0]*36
mean_post = [0]*36

for i in range(36):
    mY.append(i)
    alt_ds[i] = str(i) + 'km_ds'
    alt[i] = str(i) + 'km'

    uc[i] = uccle
    # ucm neede to calculate the monthly means, >0 is neded to eliminate -9999 values
    # ucm[i] = uccle[uccle[alt[i]]>0]
    #uc[alt[i]] = uccle[alt[i]]

    #uct[i] = uc[i].loc['1992-11-15':'20178-12-15']
    uct[i] = uc[i]

    # uct_pre[i] = ucm[i].loc['1977-02-01':'1996-12-01']
    # uct_post[i] = ucm[i].loc['2000-02-01':'2017-06-01']

    predictors, uct[i] = pd.DataFrame.align(predictors, uct[i], axis=0)

    uY[i] = uct[i][alt_ds[i]].values

    # mean_pre[i] = np.nanmean(uct_pre[i][alt[i]].values)
    # mean_post[i] = np.nanmean(uct_post[i][alt[i]].values)

    uX[i] = predictors.values
    
    print(i, uX[i])

    regression_output[i] = mzm_regression(uX[i], uY[i])
    param_list[i] = dict(zip(list(predictors), regression_output[i]['gls_results'].params))
    error_list[i] = dict(zip(list(predictors), regression_output[i]['gls_results'].bse))

    ut[i] = uct[i].index
    ptitle = str(alt[i])
    pname =str(alt[i])

    if (i == 24): print(i, len(ut[i]), len(uY[i]), len(regression_output[i]))


    trend_pre[i] = param_list[i]['linear_pre']
    trend_pre_err[i] = error_list[i]['linear_pre']
    trend_post[i] = param_list[i]['linear_post']
    trend_post_err[i] = error_list[i]['linear_post']

    print('pre', trend_pre[i], 'post', trend_post[i])


    # for % in decade for relative montly anamoly

    trend_pre[i] = trend_pre[i] * 100
    trend_pre_err[i] = 2 * trend_pre_err[i] * 100
    trend_post[i] = trend_post[i] * 100
    trend_post_err[i] = 2 * trend_post_err[i] *100

['date', '0km_ds', '1km_ds', '2km_ds', '3km_ds', '4km_ds', '5km_ds', '6km_ds', '7km_ds', '8km_ds', '9km_ds', '10km_ds', '11km_ds', '12km_ds', '13km_ds', '14km_ds', '15km_ds', '16km_ds', '17km_ds', '18km_ds', '19km_ds', '20km_ds', '21km_ds', '22km_ds', '23km_ds', '24km_ds', '25km_ds', '26km_ds', '27km_ds', '28km_ds', '29km_ds', '30km_ds', '31km_ds', '32km_ds', '33km_ds', '34km_ds', '35km_ds']
['0km_ds', '1km_ds', '2km_ds', '3km_ds', '4km_ds', '5km_ds', '6km_ds', '7km_ds', '8km_ds', '9km_ds', '10km_ds', '11km_ds', '12km_ds', '13km_ds', '14km_ds', '15km_ds', '16km_ds', '17km_ds', '18km_ds', '19km_ds', '20km_ds', '21km_ds', '22km_ds', '23km_ds', '24km_ds', '25km_ds', '26km_ds', '27km_ds', '28km_ds', '29km_ds', '30km_ds', '31km_ds', '32km_ds', '33km_ds', '34km_ds', '35km_ds']
0 [['1985-09' -1.1334935008932419 0.0 ..., 0.5191123024038149
  -1.0267837645863471 -1.0883187133704455]
 ['1985-10' -1.125279779872277 0.0 ..., 0.14919635679525795
  -1.1437135017370603 -1.0029589769547185]
 [nan nan 

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''